# MOSFETs and IGBTs

This notebook covers power semiconductor modeling in Pulsim - MOSFETs and IGBTs.

## Contents
1. MOSFET Basics and Level 1 Model
2. MOSFET Parameters
3. Power MOSFET (Rds_on) Model
4. Body Diode and Parasitic Capacitances
5. IGBT Model
6. MOSFET vs IGBT Comparison
7. Device Library

In [ ]:
import pulsim
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = [12, 5]
plt.rcParams['font.size'] = 11

## 1. MOSFET Basics and Level 1 Model

The Level 1 MOSFET model (Shichman-Hodges) defines three operating regions:

### Cutoff Region ($V_{GS} < V_{th}$)
$$I_D = 0$$

### Linear/Triode Region ($V_{GS} > V_{th}$ and $V_{DS} < V_{GS} - V_{th}$)
$$I_D = K_p \frac{W}{L} \left[ (V_{GS} - V_{th}) V_{DS} - \frac{V_{DS}^2}{2} \right] (1 + \lambda V_{DS})$$

### Saturation Region ($V_{GS} > V_{th}$ and $V_{DS} \geq V_{GS} - V_{th}$)
$$I_D = \frac{K_p}{2} \frac{W}{L} (V_{GS} - V_{th})^2 (1 + \lambda V_{DS})$$

Where:
- $K_p$ = Transconductance parameter (A/V²)
- $W/L$ = Channel width/length ratio
- $V_{th}$ = Threshold voltage
- $\lambda$ = Channel-length modulation

In [ ]:
# MOSFET Level 1 equations
def mosfet_level1(Vgs, Vds, Vth, Kp, W, L, lambda_):
    """Calculate MOSFET drain current using Level 1 model."""
    Kp_eff = Kp * W / L
    
    if Vgs <= Vth:
        # Cutoff
        return 0.0
    elif Vds < Vgs - Vth:
        # Linear/Triode
        return Kp_eff * ((Vgs - Vth) * Vds - Vds**2 / 2) * (1 + lambda_ * Vds)
    else:
        # Saturation
        return (Kp_eff / 2) * (Vgs - Vth)**2 * (1 + lambda_ * Vds)

# Vectorized version
mosfet_level1_vec = np.vectorize(mosfet_level1)

In [ ]:
# Plot MOSFET output characteristics (Id vs Vds)
Vth = 2.0     # Threshold voltage
Kp = 20e-6    # Transconductance
W = 100e-6    # Width
L = 10e-6     # Length
lambda_ = 0.02

Vds = np.linspace(0, 10, 500)
Vgs_values = [3, 4, 5, 6, 7]

plt.figure(figsize=(12, 6))

for Vgs in Vgs_values:
    Id = mosfet_level1_vec(Vgs, Vds, Vth, Kp, W, L, lambda_)
    plt.plot(Vds, Id * 1e3, linewidth=2, label=f'Vgs = {Vgs}V')
    
    # Mark transition to saturation
    Vds_sat = Vgs - Vth
    if Vds_sat > 0:
        Id_sat = mosfet_level1(Vgs, Vds_sat, Vth, Kp, W, L, lambda_)
        plt.plot(Vds_sat, Id_sat * 1e3, 'ko', markersize=6)

plt.xlabel('Vds (V)')
plt.ylabel('Id (mA)')
plt.title('MOSFET Output Characteristics (Level 1 Model)')
plt.legend()
plt.grid(True)
plt.xlim([0, 10])
plt.ylim([0, 30])
plt.tight_layout()
plt.show()

print("Black dots indicate transition from linear to saturation region")

In [ ]:
# Plot transfer characteristic (Id vs Vgs)
Vgs = np.linspace(0, 8, 500)
Vds_values = [2, 5, 10]

plt.figure(figsize=(12, 6))

for Vds in Vds_values:
    Id = mosfet_level1_vec(Vgs, Vds, Vth, Kp, W, L, lambda_)
    plt.plot(Vgs, Id * 1e3, linewidth=2, label=f'Vds = {Vds}V')

plt.axvline(x=Vth, color='r', linestyle='--', label=f'Vth = {Vth}V')
plt.xlabel('Vgs (V)')
plt.ylabel('Id (mA)')
plt.title('MOSFET Transfer Characteristic')
plt.legend()
plt.grid(True)
plt.xlim([0, 8])
plt.tight_layout()
plt.show()

## 2. MOSFET Parameters

```python
MOSFETParams:
    type = MOSFETType.NMOS  # NMOS or PMOS
    
    # Level 1 parameters
    vth = 2.0       # Threshold voltage (V)
    kp = 20e-6      # Transconductance parameter (A/V²)
    lambda_ = 0.0   # Channel-length modulation (1/V)
    w = 100e-6      # Channel width (m)
    l = 10e-6       # Channel length (m)
    
    # Body diode
    body_diode = False
    is_body = 1e-14   # Diode saturation current
    n_body = 1.0      # Diode ideality factor
    
    # Parasitic capacitances
    cgs = 0.0         # Gate-source capacitance (F)
    cgd = 0.0         # Gate-drain capacitance (F)
    cds = 0.0         # Drain-source capacitance (F)
    
    # Power MOSFET simplified model
    rds_on = 0.0      # If > 0, use simple switch model
    rds_off = 1e9     # Off-state resistance
```

In [ ]:
# Create MOSFET with Level 1 parameters
mosfet = pulsim.MOSFETParams()
mosfet.type = pulsim.MOSFETType.NMOS
mosfet.vth = 2.0
mosfet.kp = 20e-6
mosfet.lambda_ = 0.02
mosfet.w = 1.0      # Normalized
mosfet.l = 1.0

print("MOSFET Level 1 Parameters:")
print(f"  Type: {'NMOS' if mosfet.type == pulsim.MOSFETType.NMOS else 'PMOS'}")
print(f"  Vth: {mosfet.vth} V")
print(f"  Kp: {mosfet.kp*1e6:.1f} µA/V²")
print(f"  Kp_eff (Kp × W/L): {mosfet.kp_effective()*1e6:.1f} µA/V²")
print(f"  λ: {mosfet.lambda_} 1/V")

In [ ]:
# NMOS vs PMOS comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# NMOS
Vgs = np.linspace(0, 10, 500)
Vth_n = 2.0
Id_nmos = mosfet_level1_vec(Vgs, 5, Vth_n, 20e-6, 1, 1, 0.02)

axes[0].plot(Vgs, Id_nmos * 1e3, 'b-', linewidth=2)
axes[0].axvline(x=Vth_n, color='r', linestyle='--', label=f'Vth = {Vth_n}V')
axes[0].set_xlabel('Vgs (V)')
axes[0].set_ylabel('Id (mA)')
axes[0].set_title('NMOS Transfer Characteristic')
axes[0].legend()
axes[0].grid(True)

# PMOS (note: Vgs and Vth are negative)
Vgs_p = np.linspace(-10, 0, 500)
Vth_p = -2.0
# For PMOS: Id flows when Vgs < Vth (more negative)
Id_pmos = -mosfet_level1_vec(-Vgs_p, 5, -Vth_p, 10e-6, 1, 1, 0.02)

axes[1].plot(Vgs_p, -Id_pmos * 1e3, 'r-', linewidth=2)
axes[1].axvline(x=Vth_p, color='b', linestyle='--', label=f'Vth = {Vth_p}V')
axes[1].set_xlabel('Vgs (V)')
axes[1].set_ylabel('|Id| (mA)')
axes[1].set_title('PMOS Transfer Characteristic')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.show()

## 3. Power MOSFET (Rds_on) Model

For power electronics, we often use a simplified model based on datasheet parameters:

- **Rds_on**: On-state drain-source resistance
- Simple switch behavior controlled by Vgs vs Vth

$$R_{DS} = \begin{cases} R_{DS(on)} & \text{if } V_{GS} > V_{th} \\ R_{DS(off)} & \text{if } V_{GS} \leq V_{th} \end{cases}$$

In [ ]:
# Power MOSFET with Rds_on model
power_mosfet = pulsim.MOSFETParams()
power_mosfet.type = pulsim.MOSFETType.NMOS
power_mosfet.vth = 4.0          # Gate threshold
power_mosfet.rds_on = 0.044     # 44mΩ (IRF540N)
power_mosfet.rds_off = 1e9      # 1GΩ
power_mosfet.body_diode = True
power_mosfet.is_body = 1e-10
power_mosfet.n_body = 1.3

print("Power MOSFET Parameters (IRF540N-like):")
print(f"  Vth: {power_mosfet.vth} V")
print(f"  Rds_on: {power_mosfet.rds_on*1000:.1f} mΩ")
print(f"  Body diode: {'Yes' if power_mosfet.body_diode else 'No'}")

In [ ]:
# Simulate power MOSFET as switch
netlist = '''
{
  "name": "MOSFET Switch Test",
  "components": [
    {"type": "V", "name": "Vdd", "nodes": ["vdd", "0"], "value": 12},
    {"type": "V", "name": "Vgate", "nodes": ["gate", "0"],
     "waveform": {"type": "pulse", "v1": 0, "v2": 10, "td": 0.1e-3,
                  "tr": 1e-6, "tf": 1e-6, "pw": 0.4e-3, "period": 1e-3}},
    {"type": "R", "name": "Rload", "nodes": ["vdd", "drain"], "value": 10},
    {"type": "M", "name": "M1", "nodes": ["drain", "gate", "0"],
     "params": {"type": "nmos", "vth": 4.0, "rds_on": 0.044}}
  ]
}
'''

circuit = pulsim.parse_netlist_string(netlist)

options = pulsim.SimulationOptions()
options.tstop = 2e-3
options.dt = 1e-6

result = pulsim.simulate(circuit, options)
data = result.to_dict()

time_ms = np.array(data['time']) * 1e3

fig, axes = plt.subplots(3, 1, figsize=(12, 8), sharex=True)

# Gate voltage
axes[0].plot(time_ms, data['signals']['V(gate)'], 'g-', linewidth=1.5)
axes[0].axhline(y=4.0, color='r', linestyle='--', label='Vth = 4V')
axes[0].set_ylabel('Vgate (V)')
axes[0].set_title('MOSFET as Switch')
axes[0].legend()
axes[0].grid(True)

# Drain voltage
axes[1].plot(time_ms, data['signals']['V(drain)'], 'b-', linewidth=1.5)
axes[1].set_ylabel('Vdrain (V)')
axes[1].grid(True)

# Drain current (Vdd - Vdrain) / Rload
v_drain = np.array(data['signals']['V(drain)'])
i_drain = (12 - v_drain) / 10
axes[2].plot(time_ms, i_drain, 'r-', linewidth=1.5)
axes[2].set_xlabel('Time (ms)')
axes[2].set_ylabel('Idrain (A)')
axes[2].grid(True)

plt.tight_layout()
plt.show()

# Calculate on-state values
on_mask = np.array(data['signals']['V(gate)']) > 5
v_ds_on = np.mean(v_drain[on_mask])
i_d_on = np.mean(i_drain[on_mask])
rds_measured = v_ds_on / i_d_on

print(f"\nOn-state measurements:")
print(f"  Vds: {v_ds_on*1000:.1f} mV")
print(f"  Id: {i_d_on:.3f} A")
print(f"  Rds_on (measured): {rds_measured*1000:.1f} mΩ")

## 4. Body Diode and Parasitic Capacitances

Power MOSFETs have:
- **Body diode**: Intrinsic diode from source to drain
- **Parasitic capacitances**: Cgs, Cgd (Miller), Cds

```
      Drain
        │
    ┌───┼───┐
    │   │   │
   Cgd  │  Cds
    │   │   │
Gate────┤   ├──▶│── (body diode)
    │   │   │
   Cgs  │   │
    │   │   │
    └───┼───┘
        │
      Source
```

In [ ]:
# MOSFET with body diode - freewheeling in buck converter
netlist = '''
{
  "name": "MOSFET Body Diode Test",
  "components": [
    {"type": "V", "name": "Vin", "nodes": ["vin", "0"], "value": 12},
    {"type": "V", "name": "Vpwm", "nodes": ["pwm", "0"],
     "waveform": {"type": "pwm", "v_off": 0, "v_on": 10, "frequency": 100e3, "duty": 0.5}},
    
    {"type": "M", "name": "M_high", "nodes": ["vin", "pwm", "sw"],
     "params": {"type": "nmos", "vth": 2.0, "rds_on": 0.044, "body_diode": false}},
    
    {"type": "M", "name": "M_low", "nodes": ["sw", "0", "0"],
     "params": {"type": "nmos", "vth": 2.0, "rds_on": 0.044, "body_diode": true,
                "is_body": 1e-10, "n_body": 1.3}},
    
    {"type": "L", "name": "L1", "nodes": ["sw", "vout"], "value": 100e-6},
    {"type": "C", "name": "C1", "nodes": ["vout", "0"], "value": 100e-6},
    {"type": "R", "name": "Rload", "nodes": ["vout", "0"], "value": 6}
  ]
}
'''

circuit = pulsim.parse_netlist_string(netlist)

options = pulsim.SimulationOptions()
options.tstop = 100e-6
options.dt = 10e-9

result = pulsim.simulate(circuit, options)
data = result.to_dict()

time_us = np.array(data['time']) * 1e6

fig, axes = plt.subplots(2, 1, figsize=(12, 6), sharex=True)

axes[0].plot(time_us, data['signals']['V(sw)'], 'b-', linewidth=1)
axes[0].set_ylabel('V(sw) (V)')
axes[0].set_title('Switch Node Voltage (Body Diode conducts when V(sw) < 0)')
axes[0].axhline(y=0, color='r', linestyle='--', alpha=0.5)
axes[0].grid(True)

axes[1].plot(time_us, data['signals']['V(vout)'], 'r-', linewidth=1.5)
axes[1].set_xlabel('Time (µs)')
axes[1].set_ylabel('V(out) (V)')
axes[1].set_title('Output Voltage')
axes[1].grid(True)

plt.tight_layout()
plt.show()

print("When high-side turns off, inductor current flows through low-side body diode")
print("causing V(sw) to go slightly negative (~0.7V body diode drop)")

In [ ]:
# MOSFET with parasitic capacitances
mosfet_caps = pulsim.MOSFETParams()
mosfet_caps.type = pulsim.MOSFETType.NMOS
mosfet_caps.vth = 4.0
mosfet_caps.rds_on = 0.044

# IRF540N typical capacitances
mosfet_caps.cgs = 2500e-12   # 2500pF
mosfet_caps.cgd = 300e-12    # 300pF (Miller capacitance)
mosfet_caps.cds = 500e-12    # 500pF

print("MOSFET Parasitic Capacitances (IRF540N):")
print(f"  Cgs: {mosfet_caps.cgs*1e12:.0f} pF")
print(f"  Cgd: {mosfet_caps.cgd*1e12:.0f} pF (Miller cap)")
print(f"  Cds: {mosfet_caps.cds*1e12:.0f} pF")
print(f"  Ciss = Cgs + Cgd: {(mosfet_caps.cgs + mosfet_caps.cgd)*1e12:.0f} pF")
print(f"  Coss = Cds + Cgd: {(mosfet_caps.cds + mosfet_caps.cgd)*1e12:.0f} pF")
print(f"  Crss = Cgd: {mosfet_caps.cgd*1e12:.0f} pF")

## 5. IGBT Model

IGBTs combine MOSFET gate control with bipolar current handling:

```python
IGBTParams:
    vth = 5.0       # Gate threshold voltage (V)
    vce_sat = 2.0   # Collector-emitter saturation voltage (V)
    rce_on = 0.01   # On-state resistance (Ω)
    rce_off = 1e9   # Off-state resistance (Ω)
    
    # Switching times
    tf = 0.0        # Fall time (s)
    tr = 0.0        # Rise time (s)
    
    # Input capacitance
    cies = 0.0      # Input capacitance (F)
    
    # Anti-parallel diode
    body_diode = True
    is_diode = 1e-12
    n_diode = 1.0
    vf_diode = 0.7
```

In [ ]:
# IGBT I-V characteristic model
def igbt_iv(Vge, Vce, Vth, Vce_sat, Rce_on):
    """Simplified IGBT model: Vce = Vce_sat + Ic * Rce_on when on."""
    if Vge <= Vth:
        return 0.0  # Off
    else:
        # Ic = (Vce - Vce_sat) / Rce_on, but clamped
        if Vce < Vce_sat:
            return (Vce / Vce_sat) * 10  # Ramp up region
        else:
            return (Vce - Vce_sat) / Rce_on

igbt_iv_vec = np.vectorize(igbt_iv)

# Plot IGBT output characteristics
Vce = np.linspace(0, 10, 500)
Vge_values = [6, 8, 10, 12, 15]

plt.figure(figsize=(12, 6))

for Vge in Vge_values:
    Ic = igbt_iv_vec(Vge, Vce, Vth=5.0, Vce_sat=1.5, Rce_on=0.02)
    Ic = np.minimum(Ic, 100)  # Limit for plot
    plt.plot(Vce, Ic, linewidth=2, label=f'Vge = {Vge}V')

plt.axvline(x=1.5, color='r', linestyle='--', alpha=0.5, label='Vce_sat')
plt.xlabel('Vce (V)')
plt.ylabel('Ic (A)')
plt.title('IGBT Output Characteristics')
plt.legend()
plt.grid(True)
plt.xlim([0, 10])
plt.ylim([0, 80])
plt.tight_layout()
plt.show()

print("Note: IGBT has saturation voltage Vce_sat (~1.5-2V) even at high currents")

In [ ]:
# Simulate IGBT in H-bridge
netlist = '''
{
  "name": "IGBT H-Bridge",
  "components": [
    {"type": "V", "name": "Vdc", "nodes": ["vdc", "0"], "value": 400},
    
    {"type": "V", "name": "Vpwm_h", "nodes": ["pwm_h", "0"],
     "waveform": {"type": "pwm", "v_off": 0, "v_on": 15, "frequency": 10e3, "duty": 0.5}},
    {"type": "V", "name": "Vpwm_l", "nodes": ["pwm_l", "0"],
     "waveform": {"type": "pwm", "v_off": 0, "v_on": 15, "frequency": 10e3, "duty": 0.5,
                  "complementary": true, "dead_time": 1e-6}},
    
    {"type": "Q", "name": "Q1", "nodes": ["vdc", "pwm_h", "out"],
     "params": {"vth": 5.0, "vce_sat": 1.5, "rce_on": 0.02, "body_diode": true}},
    {"type": "Q", "name": "Q2", "nodes": ["out", "pwm_l", "0"],
     "params": {"vth": 5.0, "vce_sat": 1.5, "rce_on": 0.02, "body_diode": true}},
    
    {"type": "R", "name": "Rload", "nodes": ["out", "mid"], "value": 10},
    {"type": "L", "name": "Lload", "nodes": ["mid", "0"], "value": 10e-3}
  ]
}
'''

circuit = pulsim.parse_netlist_string(netlist)

options = pulsim.SimulationOptions()
options.tstop = 1e-3
options.dt = 100e-9

result = pulsim.simulate(circuit, options)
data = result.to_dict()

time_ms = np.array(data['time']) * 1e3

fig, axes = plt.subplots(2, 1, figsize=(12, 6), sharex=True)

axes[0].plot(time_ms, data['signals']['V(out)'], 'b-', linewidth=1)
axes[0].set_ylabel('V(out) (V)')
axes[0].set_title('IGBT Half-Bridge Output (400V DC Bus)')
axes[0].grid(True)

# Current through load
v_out = np.array(data['signals']['V(out)'])
v_mid = np.array(data['signals'].get('V(mid)', np.zeros_like(v_out)))
i_load = (v_out - v_mid) / 10

axes[1].plot(time_ms, i_load, 'r-', linewidth=1.5)
axes[1].set_xlabel('Time (ms)')
axes[1].set_ylabel('Load Current (A)')
axes[1].set_title('Inductive Load Current')
axes[1].grid(True)

plt.tight_layout()
plt.show()

## 6. MOSFET vs IGBT Comparison

| Feature | MOSFET | IGBT |
|---------|--------|------|
| Gate control | Voltage | Voltage |
| On-state | Rds_on × Id | Vce_sat + Rce_on × Ic |
| Low current loss | Lower (resistive) | Higher (Vce_sat) |
| High current loss | Higher (I²R) | Lower (linear) |
| Switching speed | Faster | Slower (tail current) |
| Voltage rating | Typically < 600V | Up to 6.5kV |
| Best for | High frequency, low V | High voltage, high power |

In [ ]:
# Compare conduction losses: MOSFET vs IGBT
I = np.linspace(0, 50, 500)

# MOSFET: P = I² × Rds_on
Rds_on = 0.044  # IRF540N
P_mosfet = I**2 * Rds_on

# IGBT: P = Vce_sat × I + I² × Rce_on
Vce_sat = 1.5
Rce_on = 0.02
P_igbt = Vce_sat * I + I**2 * Rce_on

# Find crossover point
crossover_idx = np.argmin(np.abs(P_mosfet - P_igbt))
I_crossover = I[crossover_idx]

plt.figure(figsize=(12, 6))
plt.plot(I, P_mosfet, 'b-', linewidth=2, label=f'MOSFET (Rds_on = {Rds_on*1000:.0f}mΩ)')
plt.plot(I, P_igbt, 'r-', linewidth=2, label=f'IGBT (Vce_sat = {Vce_sat}V, Rce = {Rce_on*1000:.0f}mΩ)')
plt.axvline(x=I_crossover, color='g', linestyle='--', 
            label=f'Crossover: {I_crossover:.1f}A')

plt.xlabel('Current (A)')
plt.ylabel('Conduction Loss (W)')
plt.title('Conduction Losses: MOSFET vs IGBT')
plt.legend()
plt.grid(True)
plt.xlim([0, 50])
plt.ylim([0, 150])
plt.tight_layout()
plt.show()

print(f"Below {I_crossover:.0f}A: IGBT has higher losses (due to Vce_sat)")
print(f"Above {I_crossover:.0f}A: MOSFET has higher losses (I²R dominates)")

## 7. Device Library

Pulsim includes pre-defined models:

### MOSFETs
| Function | Device | Vds | Rds_on | Notes |
|----------|--------|-----|--------|-------|
| `mosfet_IRF540N()` | IRF540N | 100V | 44mΩ | Standard power MOSFET |
| `mosfet_IRFZ44N()` | IRFZ44N | 55V | 17.5mΩ | Low Rds_on |
| `mosfet_IRF9540()` | IRF9540 | -100V | 117mΩ | P-channel |
| `mosfet_BSC0902NS()` | BSC0902NS | 30V | 2.1mΩ | High efficiency |
| `mosfet_EPC2001C()` | EPC2001C | 100V | 4mΩ | GaN FET |

### IGBTs
| Function | Device | Vce | Ic | Notes |
|----------|--------|-----|-----|-------|
| `igbt_IRG4PC40UD()` | IRG4PC40UD | 600V | 40A | General purpose |
| `igbt_IRG4BC30KD()` | IRG4BC30KD | 600V | 30A | High speed |
| `igbt_IKW40N120H3()` | IKW40N120H3 | 1200V | 40A | High voltage |

In [ ]:
# Library MOSFETs comparison
mosfets = {
    'IRF540N': {'rds_on': 0.044, 'vds_max': 100, 'cgs': 2500e-12},
    'IRFZ44N': {'rds_on': 0.0175, 'vds_max': 55, 'cgs': 1600e-12},
    'BSC0902NS': {'rds_on': 0.0021, 'vds_max': 30, 'cgs': 2800e-12},
    'EPC2001C (GaN)': {'rds_on': 0.004, 'vds_max': 100, 'cgs': 150e-12},
}

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Rds_on comparison
names = list(mosfets.keys())
rds_values = [m['rds_on'] * 1000 for m in mosfets.values()]
colors = plt.cm.Blues(np.linspace(0.4, 0.9, len(names)))

axes[0].barh(names, rds_values, color=colors)
axes[0].set_xlabel('Rds_on (mΩ)')
axes[0].set_title('MOSFET Rds_on Comparison')
for i, v in enumerate(rds_values):
    axes[0].text(v + 0.5, i, f'{v:.1f}', va='center')

# FOM: Rds_on × Qg (approximated by Rds_on × Cgs)
fom = [m['rds_on'] * m['cgs'] * 1e12 for m in mosfets.values()]  # mΩ·pF

axes[1].barh(names, fom, color=colors)
axes[1].set_xlabel('FOM: Rds_on × Cgs (mΩ·pF)')
axes[1].set_title('Figure of Merit (lower is better)')
for i, v in enumerate(fom):
    axes[1].text(v + 1, i, f'{v:.0f}', va='center')

plt.tight_layout()
plt.show()

print("\nGaN (EPC2001C) has best FOM due to very low capacitance")

In [ ]:
# IGBT comparison
igbts = {
    'IRG4PC40UD': {'vce_sat': 1.5, 'vce_max': 600, 'tf': 150e-9},
    'IRG4BC30KD': {'vce_sat': 1.65, 'vce_max': 600, 'tf': 60e-9},
    'IKW40N120H3': {'vce_sat': 1.95, 'vce_max': 1200, 'tf': 130e-9},
}

print("IGBT Library Comparison:")
print("-" * 60)
print(f"{'Device':<15} {'Vce_max':<10} {'Vce_sat':<10} {'Fall Time':<15}")
print("-" * 60)
for name, params in igbts.items():
    print(f"{name:<15} {params['vce_max']:<10}V {params['vce_sat']:<10}V {params['tf']*1e9:<15.0f}ns")

print("\nTrade-offs:")
print("  - IRG4PC40UD: Best Vce_sat, moderate speed")
print("  - IRG4BC30KD: Fastest switching for resonant converters")
print("  - IKW40N120H3: Highest voltage rating for grid applications")

## Summary

### MOSFET Models

| Model | Parameters | Use Case |
|-------|------------|----------|
| **Level 1** | Vth, Kp, W, L, λ | Accurate I-V curves |
| **Power (Rds_on)** | Vth, Rds_on | Simple switching analysis |
| **Full** | + Cgs, Cgd, Cds, body diode | Transient/loss analysis |

### IGBT Model

- Vce_sat + resistive model for conduction
- Switching times (tr, tf) for loss calculation
- Anti-parallel diode for freewheeling

### Selection Guide

| Application | Recommended |
|-------------|-------------|
| Low voltage (<100V), high frequency | MOSFET |
| High current (>30A), low frequency | IGBT |
| High voltage (>600V) | IGBT |
| High efficiency DC-DC | GaN/SiC MOSFET |

**Next:** [AC Analysis and Bode Plots](08_ac_analysis.ipynb)